In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os
import missingno as msno

In [2]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



In [13]:
# Carregando DataFrames das edições do censo da educação superior.
censo_23 = 'censo_2023.csv'
file_ies = 'censo_ies/MICRODADOS_ED_SUP_IES_2023.CSV'

In [4]:
pd.set_option('display.max_columns', 200)

In [5]:
pd.set_option('display.max_rows', 500)

In [6]:
pd.set_option('display.max_colwidth', 100)

In [7]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_23, 'rb') as f1:
    raw_data = f1.read(1000000)

In [8]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

codificação detectada: ISO-8859-1 (confiança: 0.73%)


In [9]:
df_23 = pd.read_csv(censo_23, sep=';', encoding=cod_23, low_memory=False, decimal=',')
df_23.shape

(671610, 194)

In [10]:
cols = ['CO_IES', 'NO_REGIAO_IES', 'CO_REGIAO_IES', 'NO_UF_IES', 'SG_UF_IES', 'CO_UF_IES', 'NO_MUNICIPIO_IES', 'CO_MUNICIPIO_IES', 'IN_CAPITAL_IES']

In [11]:
ies_23 = pd.read_csv(file_ies, sep=';', encoding=cod_23, low_memory=False, decimal=';', usecols=cols)
ies_23.shape

(2580, 9)

In [21]:
(df_23.isna().mean()*100).round(2)

NU_ANO_CENSO                    0.00
CO_IES                          0.00
NO_CURSO                        0.00
CO_CURSO                        0.00
TP_GRAU_ACADEMICO               0.00
IN_GRATUITO                     0.00
TP_MODALIDADE_ENSINO            0.00
TP_NIVEL_ACADEMICO              0.00
NO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_ESPECIFICA         0.00
NO_CINE_AREA_ESPECIFICA         0.00
CO_CINE_AREA_DETALHADA          0.00
NO_CINE_AREA_DETALHADA          0.00
NO_REGIAO                       1.68
CO_REGIAO                       0.00
NO_UF                           1.68
SG_UF                           1.68
CO_UF                           0.00
NO_MUNICIPIO                    1.68
CO_MUNICIPIO                    0.00
IN_CAPITAL                      0.00
TP_DIMENSAO                     0.00
TP_ORGANIZACAO_ACADEMICA        0.00
TP_REDE                         0.00
TP_CATEGORIA_ADMINISTRATIVA     0.00
QT_CURSO                        0.00
Q

In [43]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_23.columns if col.startswith('QT_')]
len(colunas_quantidade)

168

In [44]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
for col in colunas_quantidade:
    df_23[col] = df_23[col].fillna(0)

# Solução alternativa usando .loc
# df_11.loc[:,colunas_quantidade] = df_11[colunas_quantidade].fillna(0)

Preenchendo os valores ausentes da coluna **'TP_GRAU_ACADEMICO'**

In [45]:
# 1. vou criar uma série (mascara booleana) (True onde é nan)
mask_na = df_23['TP_GRAU_ACADEMICO'].isna()

In [46]:
# 2. Aplicar a mascara boleanda para selecionar apenas as linhas com valor True
g_acadna = df_23[mask_na]

Para começar a preencher os valores ausentes da coluna 'TP_GRAU_ACADEMICO', algumas decisões de análise precisarão ser tomadas.

Para reduzir o número de valores ausentes na coluna 'TP_NIVEL_ACADEMICO'

In [47]:
# 3. Verificando os valores únicos da variável 'TP_NIVEL_ACADEMICO'
g_acadna['TP_NIVEL_ACADEMICO'].unique()

array([1, 2], dtype=int64)

In [48]:
# 4. Criando uma mascara booleanda apenas com os cursos Sequencial de formação especifica.
mask_cs = g_acadna['TP_NIVEL_ACADEMICO'] == 2 # Sequencial de formação específica

In [49]:
# 5. Aplicando a mascara booleana para para selecionar apenas as linha com valor True da variável 'TP_NIVEL_ACADEMICO'
c_fesp = g_acadna[mask_cs]

In [50]:
c_fesp.shape

(5, 194)

In [51]:
# 6. Verificando as incidências de valores da variável 'NO_CINE_AREA_GERAL'
c_fesp['NO_CINE_AREA_GERAL'].value_counts()

NO_CINE_AREA_GERAL
Negócios, administração e direito    3
Artes e humanidades                  1
Serviços                             1
Name: count, dtype: int64

In [52]:
# 7. Criando uma cópia do DataFrame para realizar as imputações
df_2023 = df_23.copy()

In [53]:
# 7.1 Checando as dimensões do novo DF
df_2023.shape

(671610, 194)

In [54]:
# 7.2 Verificando os valores ausentes na variável alvo.
df_2023['TP_GRAU_ACADEMICO'].isna().sum()

1720

In [55]:
# 7.3 Criando uma validação condicional utilizando as duas mascaras criadas anteriormente e atribuindo um novo valor para a variável alvo.
condition = (mask_na & mask_cs)
df_2023.loc[condition, 'TP_GRAU_ACADEMICO'] = 5 #  Sequencial de formação específica

In [56]:
# 7.4 Conferindo a redução no número valores ausentes na variável alvo
df_2023['TP_GRAU_ACADEMICO'].isna().sum()

1715

In [57]:
# 8. Criando uma nova mascara de seleção dos valores ausentes remanecentes na variável 'TP_GRAU_ACADEMICO'
mask1_na = df_2023['TP_GRAU_ACADEMICO'].isna()

In [58]:
# 8.1 Criação de um novo df filtrado pela mascara criada anteriormente
g_acadna1 = df_2023[mask1_na]

In [59]:
# 8.2 Verificação da dimensão do df olhando apenas para a quantidade de linhas
g_acadna1.shape[0]

1715

In [60]:
# 8.3 Verificando algumas amostras de linhas do DataFrame para verificar a insidência de algum padrão
g_acadna1.sample(5)

,NU_ANO_CENSO,CO_IES,NO_CURSO,CO_CURSO,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,NO_CINE_AREA_GERAL,CO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_REDE,TP_CATEGORIA_ADMINISTRATIVA,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PRETA.1,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVSOCIAL_RF.1,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
540609,2023,1042,Abi - Educação Física,5001674,NaN,0,2,1,Programas básicos,0,1,Programas básicos,11,Programas básicos,Sul,4.0,Paraná,PR,41.0,Curitiba,4106902.0,1.0,2,1,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181,69,112,0,0,147,147,20,0,126,0,0,0,34,0,0,0,63,37,28,25,25,3,0,100,10,42,4,1,24,148,51,97,0,0,0,54,27,29,19,17,2,0,87,9,29,3,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181,0,148,0,0,0,4,2,4,0,13,0,0,0,0,13,1,0,3,9,18,0,0,0,0,18,1,0,5,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,59,1,0,169,12,0,136,12,0,0,0,0,0,0,0,0,0,0,0,0,22,11,20,0,0,0,0,0
446518,2023,221,Abi - Educação Física,5001656,NaN,0,2,1,Programas básicos,0,1,Programas básicos,11,Programas básicos,Sudeste,3.0,São Paulo,SP,35.0,Junqueirópol

In [61]:
# 8.4 Analisando algumas estatísticas das variáveis não númericas do DataFrame
g_acadna1.describe(include = 'object')

,NO_CURSO,NO_CINE_AREA_GERAL,NO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_DETALHADA,NO_REGIAO,NO_UF,SG_UF,NO_MUNICIPIO
count,1715,1715,1715,1715,1689,1689,1689,1689
unique,69,2,2,2,5,27,27,1031
top,Abi - Educação Física,Programas básicos,Programas básicos,Programas básicos,Sudeste,São Paulo,SP,São Paulo
freq,1490,1714,1714,1714,843,397,397,33


In [62]:
#  8.4.1 Verificando as regiões do Brasil que não preencheram a variável ´TP_GRAU_ACADEMICO´ para o cursos diferentes de ...
# 'Sequencial de formação especifica"
g_acadna1['NO_REGIAO'].value_counts()

NO_REGIAO
Sudeste         843
Sul             330
Nordeste        279
Centro-Oeste    127
Norte           110
Name: count, dtype: int64

In [63]:
# 8.4.2 Verificando insidência de valores ausentes por tipo de Rede (Pública e Privada)
g_acadna1['TP_REDE'].value_counts()

TP_REDE
2    1461
1     254
Name: count, dtype: int64

In [35]:
# 8.4.3 Visualizando a ocorrência dos cursos para determinar quais são bacharelados por definição e quais não são.
g_acadna1['NO_CURSO'].value_counts()

NO_CURSO
Abi - Educação Física                                                                       65
Abi - Ciências Biológicas                                                                   24
Abi - Ciências Sociais                                                                      18
Abi - Geografia                                                                             16
Abi - Matemática                                                                            13
Abi - História                                                                              13
Abi - Física                                                                                13
Abi - Química                                                                               13
Abi - Filosofia                                                                             12
Abi - Letras                                                                                11
Abi - Engenharia                         

In [64]:
g_acadna1['NO_CURSO'].unique()

array(['Abi - Educação Física', 'Abi - Artes Cênicas',
       'Abi - Ciências Sociais', 'Abi - Design', 'Abi - Engenharia',
       'Abi - Filosofia', 'Abi - Geografia', 'Abi - História',
       'Abi - Letras - Língua Portuguesa', 'Abi - Letras Francês',
       'Abi - Letras Inglês', 'Abi - Matemática', 'Abi - Música',
       'Abi - Psicologia', 'Abi - Ciência Da Computação', 'Abi - Química',
       'Abi - Educação Intercultural', 'Abi - Letras', 'Computação',
       'Abi - Licenciaturas Interdisciplinares',
       'Abi - Ciências Biológicas', 'Abi - Composição E Regência',
       'Abi - Dança', 'Abi - Engenharia De Minas', 'Abi - Física',
       'Abi - Letras Vernáculas',
       'Abi - Letras Vernáculas E Língua Estrangeira Moderna',
       'Abi - Língua Estrangeira',
       'Abi - Língua Estrangeira Moderna Ou Clássica',
       'Abi - Música Popular', 'Abi - Teatro',
       'Abi - Licenciatura Intercultural', 'Abi - Artes Visuais',
       'Abi - Enfermagem', 'Abi - Física/Engenharia F

In [65]:
# 8.4.4 Criando uma mascara classificando os cursos de licenciatura e bacharelado
bacharelado = [
    'Abi - Design',
    'Abi - Engenharia',
    'Abi - Engenharia De Minas',
    'Abi - Engenharia Elétrica',
    'Abi - Ciclo Básico Materiais/Metalúrgica/Nuclear',
    'Abi - Ciência Da Computação',
    'Abi - Comunicação Social',
    'Abi - Ciências Biomédicas',
    'Abi - Economia Empresarial E Controladoria',
    'Abi - Psicologia',
    'Abi - Enfermagem',
    'Computação'
]
len(bacharelado)

12

In [66]:
# 8.4.4.2 Criando a condição lógica para definir a atribuição de valores para as outras variáveis
lic_bac = [
    'Abi - Educação Física',
    'Abi - Artes Cênicas',
    'Abi - Ciências Sociais',
    'Abi - Filosofia',
    'Abi - Geografia',
    'Abi - História',
    'Abi - Matemática',
    'Abi - Química',
    'Abi - Educação Intercultural',
    'Abi - Letras',
    'Abi - Licenciaturas Interdisciplinares',
    'Abi - Ciências Biológicas',
    'Abi - Composição E Regência',
    'Abi - Dança',
    'Abi - Física',
    'Abi - Letras Vernáculas',
    'Abi - Letras Vernáculas E Língua Estrangeira Moderna',
    'Abi - Língua Estrangeira',
    'Abi - Língua Estrangeira Moderna Ou Clássica',
    'Abi - Música Popular',
    'Abi - Teatro',
    'Abi - Licenciatura Intercultural',
    'Abi - Artes Visuais',
    'Abi - Ciências Da Natureza: Ciências E Biologia, Ciências E Física Ou Ciências E Química',
    'Abi - Ciências Exatas',
    'Abi - Ciclo Básico Composição/Regência',
    'Abi - Educação Física E Esporte',
    'Abi - Matemática Aplicada',
    'Abi - Matemática Aplicada E Computacional',
    'Abi - Física/Engenharia Física',
    'Abi - Física, Matemática/Matemática Aplicada E Computacional',
    'Abi - Letras - Língua Portuguesa',
    'Abi - Letras Francês',
    'Abi - Letras Inglês',
    'Abi - Música',
    'Abi - Letras Português',
    'Abi - Letras Alemão',
    'Abi - Letras Espanhol',
    'Abi - Letras Italiano',
    'Abi - Letras Língua Portuguesa',
    'Abi - Letras - Português - Arabe',
    'Abi - Letras - Português - Espanhol',
    'Abi - Letras - Português - Frances',
    'Abi - Letras - Português - Grego',
    'Abi - Letras - Português - Hebraico',
    'Abi - Letras - Português - Ingles',
    'Abi - Letras - Português - Italiano',
    'Abi - Letras - Português - Latim',
    'Abi - Letras - Português - Russo',
    'Abi - Letras Português - Alemão',
    'Abi - Literaturas De Língua Portuguesa',
    'Matemática - Licenciatura Ou Bacharelado',
    'Ciências Biológicas - Licenciatura Ou Bacharelado',
    'Ciências Sociais - Licenciatura Ou Bacharelado',
    'Geografia - Licenciatura Ou Bacharelado',
    'História - Licenciatura Ou Bacharelado',
    'Química - Licenciatura Ou Bacharelado'
]

In [67]:
# 8.4.4.1 Criando a condição lógica para definir a atribuição de valores
mask2 = df_2023['NO_CURSO'].isin(bacharelado)
mask3 = df_2023['TP_GRAU_ACADEMICO'].isna()
mask4 = df_2023['NO_CURSO'].isin(lic_bac)
condiction1 = (mask2 & mask3)
condiction2 = (mask4 & mask3)

In [68]:
# Conferindo número de atribuições da condiction1
condiction1.sum()

22

In [69]:
df_2023['TP_GRAU_ACADEMICO'].isna().sum()

1715

In [70]:
# 8.4.4.3 Atribuição para os registros que atendem a condição de não licenciaturas
df_2023.loc[condiction1, 'TP_GRAU_ACADEMICO'] = 1 # Bacharelado

In [71]:
df_2023['TP_GRAU_ACADEMICO'].isna().sum()

1693

In [72]:
# 8.4.4.4 Atribuição para os registros que atendem a condição de licenciaturas
df_2023.loc[condiction2, 'TP_GRAU_ACADEMICO'] = 4 # Bacharelado ou licenciaturas

In [73]:
# Verificando o número de atribuições da condiction2
condiction2.sum()

1693

In [74]:
# 9. Verificação da incidência de valores nulos na variável 'TP_GRAU_ACADEMICO'
df_2023['TP_GRAU_ACADEMICO'].isna().sum()

0

In [75]:
df_tgau = df_2023['TP_GRAU_ACADEMICO'].isna()

In [76]:
df = df_2023[df_tgau]

In [77]:
df['NO_CURSO'].unique()

array([], dtype=object)

**Tratamento das demais variáveis com valores ausentes**

['NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL']

In [78]:
(df_2023.isna().mean()*100).round(2)

NU_ANO_CENSO                    0.00
CO_IES                          0.00
NO_CURSO                        0.00
CO_CURSO                        0.00
TP_GRAU_ACADEMICO               0.00
IN_GRATUITO                     0.00
TP_MODALIDADE_ENSINO            0.00
TP_NIVEL_ACADEMICO              0.00
NO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_GERAL              0.00
CO_CINE_AREA_ESPECIFICA         0.00
NO_CINE_AREA_ESPECIFICA         0.00
CO_CINE_AREA_DETALHADA          0.00
NO_CINE_AREA_DETALHADA          0.00
NO_REGIAO                       1.64
CO_REGIAO                       1.64
NO_UF                           1.64
SG_UF                           1.64
CO_UF                           1.64
NO_MUNICIPIO                    1.64
CO_MUNICIPIO                    1.64
IN_CAPITAL                      1.64
TP_DIMENSAO                     0.00
TP_ORGANIZACAO_ACADEMICA        0.00
TP_REDE                         0.00
TP_CATEGORIA_ADMINISTRATIVA     0.00
QT_CURSO                        0.00
Q

Aplicação da função .map() para otimização **'NO_REGIAO'**

In [79]:
# 1. Verificando as dimensões do df lookup (Capastro_ies - fonte de mapeamento)
ies_23.shape

(2580, 9)

In [80]:
# 1.2 Verificando se existe dados duplicados na variável chave do df lookup
ies_23['CO_IES'].duplicated().sum()

0

In [81]:
# 2. Criação da série de mapeamento (dicionário de lookup) index= CO_IES (chave) Values = 'NO_REGIAO'
map_cat1 = ies_23.set_index('CO_IES')['NO_REGIAO_IES']
# O método .drop_duplicates() é essencial para garantir que a chave (CO_IES) seja única, evitando que o .map() falhe ou retorne valores incorretos.
#map_cat1 = map_cat1.drop_duplicates() # no caso atual, não é necessário pois a os registro do df ies_10 são 1 para 1.

In [82]:
map_cat1.head()

CO_IES
1    Centro-Oeste
2    Centro-Oeste
3        Nordeste
4           Norte
5        Nordeste
Name: NO_REGIAO_IES, dtype: object

In [83]:
# Gerando uma nova serie com os valores do mapeamento
cat_map = df_2023['CO_IES'].map(map_cat1)

In [84]:
df_2023['NO_REGIAO'] = df_2023['NO_REGIAO'].fillna(cat_map)

In [85]:
df_2023['NO_REGIAO'].isna().sum()

0

Aplicando a função na variável **'CO_REGIAO'**

In [86]:
# 1. Criação da série de mapeamentos (lookup)
map_cat2 = ies_23.set_index('CO_IES')['CO_REGIAO_IES']

In [87]:
# 2. Gerando série com os valores do mapeamento
cat_map2 = df_2023['CO_IES'].map(map_cat2) 

In [88]:
# 3. Preechimento de valores com .fillna()
df_2023['CO_REGIAO'] = df_2023['CO_REGIAO'].fillna(cat_map2)

In [89]:
# 4. Validando a aplicação
df_2023['CO_REGIAO'].isna().sum()

0

Aplicando a função na variável **'NO_UF'**

In [90]:
# 1. Criação da série de mapeamento (lookup)
map_cat3 = ies_23.set_index('CO_IES')['NO_UF_IES']

In [91]:
# 2. Gerando série com os valores do mapeamento
cat_map3 = df_2023['CO_IES'].map(map_cat3)

In [92]:
# 3. Preenchimento de valores com .fillna()
df_2023['NO_UF'] = df_2023['NO_UF'].fillna(cat_map3)

In [93]:
# 4. Validação da aplicação
df_2023['NO_UF'].isna().sum()

0

Aplicando a função na variável **'SG_UF'**

In [94]:
# 1. Criação da série de mapeamento (lookup)
map_cat4 = ies_23.set_index('CO_IES')['SG_UF_IES']

In [95]:
# 2. Gerando série com os valores do mapeamento
cat_map4 = df_2023['CO_IES'].map(map_cat4)

In [96]:
# 3. Preenchimento de valores com .fillna()
df_2023['SG_UF'] = df_2023['SG_UF'].fillna(cat_map4)

In [97]:
# 4. Validação da aplicação
df_2023['SG_UF'].isna().sum()

0

Aplicando a função na variável **'CO_UF'**

In [98]:
df_2023.CO_UF.isna().sum()

11039

In [99]:
# 1. Criação de série de mapeamento (lookup)
map_cat5 = ies_23.set_index('CO_IES')['CO_UF_IES']

In [100]:
# 2. Gerando série com os valores do mapemento
cat_map5 = df_2023['CO_IES'].map(map_cat5)

In [101]:
# 3. Preechimento de valores com .fillna()
df_2023['CO_UF'] = df_2023['CO_UF'].fillna(cat_map5)

In [102]:
# 4. Validação da aplicação
df_2023.CO_UF.isna().sum()

0

Aplicando a função na variável **'NO_MUNICIPIO'**

In [103]:
df_2023.NO_MUNICIPIO.isna().sum()

11039

In [104]:
# 1. Criação de série de mapeamento (lookup)
map_cat6 = ies_23.set_index('CO_IES')['NO_MUNICIPIO_IES']

In [105]:
# 2. Gerando série com os valores do mapeamento
cat_map6 = df_2023['CO_IES'].map(map_cat6)

In [106]:
# 3. Preenchimento de valores com fillna()
df_2023['NO_MUNICIPIO'] = df_2023['NO_MUNICIPIO'].fillna(cat_map6)

In [107]:
# 4. Validação
df_2023['NO_MUNICIPIO'].isna().sum()

0

Aplicando a função na variável **'CO_MUNICIPIO'**

In [108]:
df_2023.CO_MUNICIPIO.isna().sum()

11039

In [109]:
# 1. Criação de série de mapeamento (lookup)
map_cat7 = ies_23.set_index('CO_IES')['CO_MUNICIPIO_IES']

In [110]:
# 2. Gerando série com os valores do mapeamento
cat_map7 = df_2023['CO_IES'].map(map_cat7)

In [111]:
# 3. Preenchimento de valores com fillna()
df_2023['CO_MUNICIPIO'] = df_2023['CO_MUNICIPIO'].fillna(cat_map7)

In [112]:
# 4. Validação
df_2023['CO_MUNICIPIO'].isna().sum()

0

Aplicando a função na variável **'CO_MUNICIPIO'**

In [113]:
df_2023.IN_CAPITAL.isna().sum()

11039

In [114]:
# 1. Criação de série de mapeamento (lookup)
map_cat8 = ies_23.set_index('CO_IES')['IN_CAPITAL_IES']

In [115]:
# 2. Gerando série com os valores do mapeamento
cat_map8 = df_2023['CO_IES'].map(map_cat8)

In [116]:
# 3. Preenchimento de valores com fillna()
df_2023['IN_CAPITAL'] = df_2023['IN_CAPITAL'].fillna(cat_map8)

In [117]:
# 4. Validação
df_2023['IN_CAPITAL'].isna().sum()

0

#### Aplicação de imputação condicional para para preencher os valores ausentes da variável 'TP_REDE' 

In [ ]:
# 1. Definindo os valores da variável que serão usadas para validação
#ies_publicas = [1, 2, 3] # TP_CATEGORIA_ADMINISTRATIVA (PÚBLICA)
# 2. Definindo a condição de nulidade da variável TP_REDE
#cond_na = df_2013['TP_REDE'].isna()
# 3. Definindo a condição da coluna secundária
#cond_sec = df_2013['TP_CATEGORIA_ADMINISTRATIVA'].isin(ies_publicas)
# 4. Combinando as duas condições (NA e [1, 2, 3])
#conditions = (cond_na & cond_sec)
# 5. Aplicando a alteração (imputação condicional)
#valor = 1
#df_2013.loc[conditions, 'TP_REDE'] = valor

In [ ]:
# print(f" Aplicado em {conditions.sum()} registros")

In [ ]:
# 1. Definindo  os valores da variável que será usada para validação
#ies_privadas = [4, 5, 6, 7, 8, 9]
# 2. Condição de nulidade
#cond_na2 = df_2013['TP_REDE'].isna()
# 3. Coluna segundária
#cond_sec2 = df_2013['TP_CATEGORIA_ADMINISTRATIVA'].isin(ies_privadas)
# 4. Combinando as duas condições
#condictions2 = (cond_na2 & cond_sec2)
# 5. Aplicar a Alteração
#valor2 = 2
#df_2013.loc[condictions2, 'TP_REDE'] = valor2

In [ ]:
#print(f"Aplicado em {condictions2.sum()} registros")

In [ ]:
# Restam valores ausentes?
#df_2013['TP_REDE'].isna().sum()

In [118]:
# Existem outras variáveis com valores ausentes?
(df_2023.isna().mean()*100).round(2)

NU_ANO_CENSO                    0.0
CO_IES                          0.0
NO_CURSO                        0.0
CO_CURSO                        0.0
TP_GRAU_ACADEMICO               0.0
IN_GRATUITO                     0.0
TP_MODALIDADE_ENSINO            0.0
TP_NIVEL_ACADEMICO              0.0
NO_CINE_AREA_GERAL              0.0
CO_CINE_AREA_GERAL              0.0
CO_CINE_AREA_ESPECIFICA         0.0
NO_CINE_AREA_ESPECIFICA         0.0
CO_CINE_AREA_DETALHADA          0.0
NO_CINE_AREA_DETALHADA          0.0
NO_REGIAO                       0.0
CO_REGIAO                       0.0
NO_UF                           0.0
SG_UF                           0.0
CO_UF                           0.0
NO_MUNICIPIO                    0.0
CO_MUNICIPIO                    0.0
IN_CAPITAL                      0.0
TP_DIMENSAO                     0.0
TP_ORGANIZACAO_ACADEMICA        0.0
TP_REDE                         0.0
TP_CATEGORIA_ADMINISTRATIVA     0.0
QT_CURSO                        0.0
QT_VG_TOTAL                 

In [119]:
# Salvando em .csv
df_2023.to_csv('censo_2023.csv', sep=';', encoding=cod_23, index=False)

### Fim